<a href="https://colab.research.google.com/github/jacksontenorio8/Desafio_case/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2 
import numpy as np 
from keras.models import Sequential 
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense, Dropout 
from keras import backend as K 
from keras.optimizers import Adam 
from keras.preprocessing.image import img_to_array 
from keras.utils import to_categorical 
from keras.preprocessing.image import ImageDataGenerator 
from keras.callbacks import ModelCheckpoint 


In [ ]:
def get_data_x_and_y(data_path, width, height, depth, classes):
    
    labels = []
    data = []
    
    for filename in os.listdir(data_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            
            image = cv2.imread(os.path.join(data_path, filename))
            if depth == 1:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            image = cv2.resize(image, (width, height))
            
            image = img_to_array(image)
            
            data.append(image)
            
            labels.append(int(filename[5])-1)
    
    X = np.array(data, dtype="float32") / 255.0
    
    Y = to_categorical(labels, num_classes=classes)
    return (X, Y)

In [ ]:
def create_lenet(input_shape):
    
    
    model = Sequential()

    
    model.add(Conv2D(20, (5, 5), padding="same", input_shape=input_shape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    
    model.add(Conv2D(50, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))

    
    model.add(Dense(classes))
    model.add(Activation("softmax"))
    return model


In [ ]:
if __name__ == '__main__':
    train_path = '/content/drive/MyDrive/cats_and_dogs/train' 
    test_path = '/content/drive/MyDrive/cats_and_dogs/test' 
    models_path = "/content/drive/MyDrive/cats_and_dogs/models"
    width = 100 
    height = 100 
    depth = 1 
    classes = 2 
    epochs = 10 
    init_lr = 1e-3 
    batch_size = 32 
    input_shape = (height, width, depth) 
    save_model = os.path.join(models_path, "lenet-{epoch:02d}-{acc:.3f}-{val_acc:.3f}.model")
    os.makedirs(models_path, exist_ok=True)

    (trainX, trainY) = get_data_x_and_y(train_path, width, height, depth, classes)
    (testX, testY) = get_data_x_and_y(test_path, width, height, depth, classes)

    aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                fill_mode="nearest")

    model = create_lenet(input_shape)

    opt = Adam(lr=init_lr, decay=init_lr / epochs)

    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    model.summary()

    print("\n training network")

    checkpoint1 = ModelCheckpoint(save_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    checkpoint2 = ModelCheckpoint(save_model, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1,checkpoint2]

    H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
            validation_data=(testX, testY), steps_per_epoch=len(trainX) // batch_size,
            epochs=epochs, verbose=1,callbacks=callbacks_list)
